In [1]:
from multi2 import *
import numpy as np

In [7]:
params = {
    'processors' : [1, 2, 4],
    'nodes' : list(range(100, 501, 100)), 
    'probability' : list(np.linspace(.1, .3, 3)),
    'blocking' : {
        'random' : random, 
        'descending chunks' : descending_degree_chunks,
        'descending within chunks' : descending_degree_within
    }
}

In [8]:
# def parallel_color_strat(G, p, blocking):
#     ''' Modified Algorithm 1 to take alternate blocking algorithms.'''
#     # Phase 1
#     # Takes blocking function 
#     Vp = blocking(G, p)
#     min_blen = min([len(block) for block in Vp])
#     batches = nodes_iter(Vp)

#     manager = Manager()
#     coloring = manager.dict()

#     for i in range(len(batches)):

#         jobs = [Process(target=color_node, args=(G, batches[i][j], coloring,)) for j in range(len(batches[i]))]
#         _ = [proc.start() for proc in jobs]
#         _ = [proc.join() for proc in jobs]
    
#     # Phase 2
#     A = []

#     for i in range(len(batches)):
#         for v in batches[i]:
#             for u in G.neighbors(v):
#                 if u in batches[i]:
#                     if coloring[u] == coloring[v]:
#                         if min(u, v) not in A:
#                             A.append(min(u, v))

#     for v in A:
#         color_node(G, v, coloring)
#     return coloring, len(A)

In [9]:
def run_tests(p, n, prob, blocking):
    final_dict = {}
    
    G = nx.gnp_random_graph(n, prob)
    
    final_dict['avg_deg'] = avg_degree(G)

    start_time = time.process_time()
    color_p, num_conflicts = parallel_color_strat(G, p, blocking)
    end_time = time.process_time()
    
    final_dict['runtime_parallel'] = end_time - start_time
    final_dict['num_conflicts'] = num_conflicts
    final_dict['chrom_num_parallel'] = max(color_p.values())

    start_time = time.process_time()
    color_g = greedy_color(G)
    end_time = time.process_time()
    
    final_dict['runtime_greedy'] = end_time - start_time
    final_dict['chrom_num_greedy'] = max(color_g.values())

    return final_dict

In [10]:
result_dict = {}
# cores, nodes, prob, blocking
for p in params['processors']:
    result_dict[p] = {}
    for n in params['nodes']:
        result_dict[p][n] = {}
        for prob in params['probability']:
            result_dict[p][n][prob] = {}
            for method in params['blocking']:
                result_dict[p][n][prob][method] = run_tests(p, n, prob, params['blocking'][method])

IndexError: index 1 is out of bounds for axis 0 with size 1